<h1 align="center" style='color: #95cbf5; background-color: #000000'>
Analyzing and visualizing the COVID-19 spread's</h1>

<p style="font-size:17px; color:black; font-weight:light; margin-left: 10px;margin-right: 10px;">
    Coronavirus infection COVID-19 (abbreviation for <span style="color:red">CO</span>rona V<span style="color:red">I</span>rus <span style="color:red">D</span>isease 20<span style="color:red">19</span>), previously coronavirus infection 2019-nCoV is a potentially severe acute respiratory infection caused by the SARS-CoV-2 virus.<br>
It is a dangerous disease that can occur both in the form of an acute respiratory viral infection of a mild course, and in a severe form, specific complications of which can include viral pneumonia, which entails acute respiratory distress syndrome or respiratory failure with the risk of death.
</p>
<p style="font-size:17px; color:black; font-weight:bold;margin-right: 10px; ">
<span style="color:red">Disclaimer</span>: this Notebook is for educational purposes only, and it <span style="color:red">NOT</span> a valid source of information for COVID-19, which is a potential threat, and you should consult the legit sources for accurate information, such as WHO...
</p>

## The model

There are four different variables in this model:

**S** the number of susceptible persons(individuals) at a given time t,not infected yet

**E** the number of Exposed persons to the infection,have the desease but don't show symptoms

**I** is the number of infected individuals at a given time t

**R** is the number of recovered persons, individuals that can not transmit virus to others. 


**N** is the total number of population

I modelled these according to the SIERD model:
	
$$
\begin{align}
\frac{dS} {dt} &= -\beta \frac{S}{N} I \\\\
\frac{dE}{dt}  &= +\beta \frac{S}{N} I -\sigma E \\\\
\frac{dI}{dt} &= +\sigma E - \gamma I \\\\
\frac{dR}{dt} &= (1-f) \gamma I \\\\
\frac{dD}{dt} &= f \gamma I
\end{align}
$$

at the beginning, i set E=1 in my model (1 exposed person, but doesnt know/show he has the virus)


In [1]:
#++++++++++++++++++
#| Covid-19 model | 
#++++++++++++++++++
#  Susceptible-Exposed-Infectious-Recovered-Deceased-Model(SEIRD)
#  S the number of susceptible persons(individuals) at a given time t,not infected yet
#  E the number of Exposed persons to the infection,have the desease but don't show symptoms
#  I is the number of infected individuals at a given time t, the persons who can 
#  transmit the virus to susceptible persons
#  R is the number of recovered persons, individuals that can not transmit virus to others
#  Assumes while recovered becoming immune to the virus
#  D is the number of deaths at time t
#  importing some useful libraries
import pandas as pd
import numpy as np
from scipy.integrate import odeint
from ipywidgets import widgets
from IPython.display import display
import plotly as pt
import plotly.graph_objs as go
#+++++++++++++++++++++++++++++++++++++++
#| ordinary differential equations(ODE)|
#+++++++++++++++++++++++++++++++++++++++
class SIERD_model():
	def __init__(self,sigma,gamma,beta,f):
		self.sigma = sigma # Rate of becoming infections after contact with the exposed(unit in [day]**-1)
		self.gamma = gamma # Rate of either dying or recovering(unit in [day]**-1) 
		self.beta = beta # Attack(contact) rate, those who can infect others(unit [day]**-1[person]**-1)
		self.f = f # fatality rate,f% of infected people will die and (1-f)% will survive(recovered)    
	def Diff_Equ(self,y_t, t):
		""" A simple model for covid spread """
		S, E, I, R, D = y_t
		Nn = S+E+I+R # Total still alive
		# I think in this model we must divide by Nn and not by N because those who still alive will influence 
		# more than the totlal population to our model(over time you will get my point i guess) 
		# The model equations
		dS_dt = -self.beta*(S/Nn)*I
		dE_dt = +self.beta*(S/Nn)*I - self.sigma*E
		dI_dt = self.sigma*E - self.gamma*I
		dR_dt = (1.0-self.f)*self.gamma*I
		dD_dt = self.f*self.gamma*I
		return [dS_dt, dE_dt, dI_dt, dR_dt, dD_dt]
def get_data_from_csv(country,number_first_cases):
	# number_first_cases is the number of confirmed cases for the first day(=1) 
	# in this country to get the dateTime and use it later on	
	# reading the csv confirmed cases file
	try:
		conf_file = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
		covid_19_conf = pd.read_csv(conf_file)
		# reading the csv death cases file
		death_file = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
		covid_19_deaths = pd.read_csv(death_file)
		# reading the csv recovered cases file
		rec_file = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
		covid_19_recov = pd.read_csv(rec_file)
	except:
		print("something went wrong during reading files from the repo!!!")
	#now lets clean up the data
	#replace every nan value,if exists, with empty string
	covid_19_conf = covid_19_conf.replace(np.nan,'',regex=True)
	covid_19_deaths = covid_19_deaths.replace(np.nan,'',regex=True)
	covid_19_recov = covid_19_recov.replace(np.nan,'',regex=True)
	# group the data by Country
	# use reset_index to leave "Country/Region" as a column not as an index for the new dataFrame
	covid_19_conf_gr = covid_19_conf.groupby("Country/Region").sum().reset_index()
	covid_19_deaths_gr = covid_19_deaths.groupby("Country/Region").sum().reset_index()
	covid_19_recov_gr = covid_19_recov.groupby("Country/Region").sum().reset_index()
	#preprocessing
	trans_covid_19_conf_cases=covid_19_conf_gr.drop(['Lat', 'Long'],axis=1).transpose()
	new_header = trans_covid_19_conf_cases.iloc[0] #grab the first row for the header
	trans_covid_19_conf_cases = trans_covid_19_conf_cases[1:] #take the data without the header row
	trans_covid_19_conf_cases.columns = new_header #set the header row as the dataFrame header
	#deaths
	trans_covid_19_deaths_cases=covid_19_deaths_gr.drop(['Lat', 'Long'],axis=1).transpose()
	trans_covid_19_deaths_cases = trans_covid_19_deaths_cases[1:] #take the data without the header row
	trans_covid_19_deaths_cases.columns = new_header
	#recovered
	trans_covid_19_recov_cases=covid_19_recov_gr.drop(['Lat', 'Long'],axis=1).transpose()
	trans_covid_19_recov_cases = trans_covid_19_recov_cases[1:] #take the data without the header row
	trans_covid_19_recov_cases.columns = new_header
	# we can compute the daily new cases 
	trans_covid_19_new_cases = trans_covid_19_conf_cases - trans_covid_19_conf_cases.shift(1)
	trans_covid_19_new_cases.iloc[0,:] = trans_covid_19_conf_cases.iloc[0,:].values # Replace nan values with first 
	#row of cumulative number of confirmed cases
	#result verification
	print("Daily number of infected cases :\n",trans_covid_19_new_cases.loc[:,country ].values)
	print("\n\nCumulative number of infected cases :\n",trans_covid_19_conf_cases.loc[:,country ].values)
	#get the index of first confirmed cases
	index_first_case = list(trans_covid_19_new_cases.loc[:,country]).index(number_first_cases)
	print("\n\n Index of first cases = "+str(index_first_case)+"\n\n")
	#test it on the array
	print(trans_covid_19_new_cases.loc[:,country ].values[index_first_case:])
	#compute active cases using sub function because the type is a pandas series
	trans_covid_19_active_cases = trans_covid_19_conf_cases.loc[:,country ].sub(trans_covid_19_recov_cases.loc[:,country],
		fill_value=0).sub(trans_covid_19_deaths_cases.loc[:,country ],fill_value=0)
	return [trans_covid_19_conf_cases.loc[:,country ],trans_covid_19_recov_cases.loc[:,country],
	trans_covid_19_deaths_cases.loc[:,country ],trans_covid_19_active_cases,index_first_case]
def Solve_And_Plot(N,sigma,gamma,f,beta,model,y_initial,t_domain,active,deaths,recov):
	Solve = odeint(model.Diff_Equ, y_initial, t_domain)
	S,E,I,R,D = [Solve[:,i] for i in range(5)]
	#sliders
	sigma_slider = widgets.FloatSlider(
		value=0.160,
		min=0.0,
		max=1.0,
		step=0.01,
		description='𝜎',
		continuous_update=False
	)
	beta_slider  = widgets.FloatSlider(
		value=0.20,
		min=0.0,
		max=1.0,
		step=0.01,
		description='𝛽',
		continuous_update=False
	)
	gamma_slider  = widgets.FloatSlider(
		value=0.020,
		min=0.0,
		max=1.0,
		step=0.002,
		description='𝛾',
		continuous_update=False
	)
	death_slider  = widgets.FloatSlider(
		value=0.130,
		min=0.0,
		max=1.0,
		step=0.01,
		description='f',
		continuous_update=False
	)
	print(active)
	#plot the model for prediction
	layout = go.Layout(title =  dict(text ='Cases',font =dict(family='Sherif',size=18,color = 'white')),
		yaxis=dict(title="number of cases", zeroline=False,showgrid = True,gridcolor='gray'),
		xaxis=dict(title="DateTime",zeroline = False,showgrid = True, tickangle = 60,gridcolor='gray'),
		paper_bgcolor='#57565e',plot_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightyellow'))
	fig = go.FigureWidget(data=[
		go.Scatter(name="Susciptible", x=t_domain,y=S,line=dict(color="#ff2a17"),mode="lines"),
		go.Scatter(name='Infected', x=t_domain,y=I,line=dict(color="black"), mode="lines"),
		go.Scatter(name='Recovered', x=t_domain, y=R,line=dict(color="green"), mode="lines"),
		go.Scatter(name='Deaths', x=t_domain,y=D,line=dict(color="yellow"), mode="lines"),
		go.Scatter(name='Recovered_data', x=np.arange(len(recov.values)),
			y=recov.values,line=dict(color="dodgerblue"), mode="lines"),
		go.Scatter(name='Deaths_data', x=np.arange(len(deaths.values)),
			y=deaths.values,line=dict(color="darkgray"), mode="lines"),
		go.Scatter(name='active_data', x=np.arange(len(active.values)),
			y=active.values,line=dict(color="blueviolet"), mode="lines")],layout=layout)
	# Change the bar mode
	fig.layout.update(barmode='group')
	def response(change):
		with fig.batch_update():
			model.sigma = sigma_slider.value
			model.gamma = gamma_slider.value
			model.beta = beta_slider.value
			model.f = death_slider.value
			fig.data[0].y=odeint(model.Diff_Equ, y_initial, t_domain)[:,0]
			fig.data[1].y=odeint(model.Diff_Equ, y_initial, t_domain)[:,2]
			fig.data[2].y=odeint(model.Diff_Equ, y_initial, t_domain)[:,3]
			fig.data[3].y=odeint(model.Diff_Equ, y_initial, t_domain)[:,4]
	sigma_slider.observe(response, names="value")
	gamma_slider.observe(response, names="value")
	beta_slider.observe(response, names="value")
	death_slider.observe(response, names="value")
	#stack the sliders horizontally
	container = widgets.HBox([sigma_slider,gamma_slider,beta_slider,death_slider])
	display(container,fig)
	response("any kinda shit for triggering the event") 
if __name__ == "__main__":
	conf_cases,recov_cases,deaths,active,index_first_case = get_data_from_csv(country = "France",number_first_cases = 2)
	print(conf_cases)
	#++++++++++++++++++++++++
	#|      Constants 		|
	#++++++++++++++++++++++++
	N =300000 # Total number of population(uninfected)
	sigma =1/6# Rate of becoming infections after contract(unit [day]**-1) after 6 days of contact 
	#the exposed person become infected by the virus
	#alpha = f*gamma    # Rate of recovery(unit [day]**-1)
	gamma = 1/120 # Rate of either dying or recovering(unit [day]**-1) i set to 120 day because there is some 
	#delay at begining for the first recovered case(42 days  for the first recovered case)
	f = 0.1# mortality rate,10% of infected people will die and 90% will survive
	beta = 0.21#Attack(contact) rate, those who can infect others(unit [day]**-1[person]**-1)
	# one person infected but doesn't have the symptoms
	y_initial = [N, 0, 1, 0, 0]
	time = 300
	t_domain  = np.linspace(0, time, time)
	model = SIERD_model(sigma,gamma,beta,f)
	Solve_And_Plot(N,sigma,gamma,f,beta,model,y_initial,t_domain,active,deaths,recov_cases)


Daily number of infected cases :
 [0 0 2 1 0 0 1 1 0 0 1 0 0 0 0 0 0 5 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 4
 20 19 43 30 61 13 84 92 276 303 177 83 575 499 0 1388 815 36 2151 1032
 1409 1846 1788 1705 1780 3880 2499 2978 3951 3851 4703 2603 4462 7657
 4922 2180 5273 4298 1912 3931 3820 3894 4309 4372 3125 26849 3682 4971
 3220 12490 2009 2 4925 2382 2775 -2188 2323 1649 1692 576 3743 3090 -2510
 756 6 1213 407 658 1104 3537 694 1284 580 312 453 802 -165 810 636 0 63
 358 882 767 251 64 21 673 358]


Cumulative number of infected cases :
 [0 0 2 3 3 3 4 5 5 5 6 6 6 6 6 6 6 11 11 11 11 11 11 11 12 12 12 12 12 12
 12 12 12 12 14 18 38 57 100 130 191 204 288 380 656 959 1136 1219 1794
 2293 2293 3681 4496 4532 6683 7715 9124 10970 12758 14463 16243 20123
 22622 25600 29551 33402 38105 40708 45170 52827 57749 59929 65202 69500
 71412 75343 79163 83057 87366 91738 94863 121712 125394 130365 133585
 146075 148084 148086 153011 155393 158168 155980 158303 159952 161644
 162220 165963 169053 166

FigureWidget({
    'data': [{'line': {'color': '#ff2a17'},
              'mode': 'lines',
              'name'…